Importing libraries


In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import sqlite3

Setting up a fuction to work along the scrap


In [3]:
class OLX:
    def __init__(self):
        pass
    def request_olx(self, url):
        headers = {
            'authority': 'www.olx.com.br',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
            'accept-language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
            'cache-control': 'max-age=0',
            'cookie': 'loginIdentifier=OTM0MDM5Y2QyNGRmZjhhZWY0YWJkYTZmNWJlODgxMWY6ZDE5ZWUzZmU1NDY2NTEwYWViNmY4YmJmYjQ0N2EyZTRjZjJjYWIxNDgzZWJiMDNhZGU5Y2ViMWQ0MGM0M2RkODExZTJhMzNlODFhZDhmNDU0M2UxNjQ0ZGVlMjdmMjNhMDgwYmJiMDhhYjczYjNjMzlkZWEzNWRlM2I1NjJhYmU3NDg4ZDY1NGI2YWQwMmUzZjlkNDhlYWI5NzkwMDhmZDU3NzFlMDc2YzBjZDU4OWIyZDg4M2ZmOWFlMGFiZDYxOTk0Y2M2M2JiOGRmMzc0YWJkNDg1ZTgxNWUwMDUxYjBhMGZlODI4YzdlMzBkMTFjZmNjZWQxMjU2NmU3ZWYyMGI5YTA3ODViYWM1NGYxOTEwODc0ZmQzMmZmNjIyMDA5; __spdt=e5285bcf701c4d23884dc33e0c279483; olx_lr_controller=1296000; session_id=undefined; nl_id=a5b4aa09-58d8-4d67-aa77-eaf1f153b893; l_id=1a067676-5ee2-4b30-af21-17c462d84820; lo_uid=1689600386301-ei64av89fmu; _tt_enable_cookie=1; _ttp=JSHTzh5YjsNsVuDCeUiBL0r4lcu; _cc_id=1637950d85e7af400608285ce846f26b; __gsas=ID=9a8a5c243a08d826:T=1689600548:RT=1689600548:S=ALNI_MaXWAE9YKbfLeSPg70ClpOl_kEdHw; ACC_LL=2|MTAyMjI0NDMyNzA5MDM1ODkxMTUw; _hjSessionUser_736533=eyJpZCI6ImE1ZTI5NjczLWE0YTMtNWU1OC05MTNhLWNlMGZiYzRjYjUzMiIsImNyZWF0ZWQiOjE2ODk2MDAzODQ2OTYsImV4aXN0aW5nIjp0cnVlfQ==; userID=4faa46c4-016c-4b2a-8e48-2f6a6fbf42dd; _gcl_au=1.1.1893538978.1697452346; _ttuu.s=1698573928613; tt.u=0100007FDB4D8864B106C5400295EF1E; r_id=5837d93a-14fa-4c5c-9152-f0310afc53b8; olx_lr_controller=1296000; _clck=lnuw98|2|fi4|0|1339; _gat_UA-70177409-2=1; _hjSession_736533=eyJpZCI6IjYzM2M2OTRhLWM4NDctNDUzNy05OGFmLTk0OGQxNWZmMTAyMiIsImMiOjE3MDQ4Mjk0MDcxMjEsInMiOjAsInIiOjAsInNiIjowfQ==; _lr_geo_location_state=PA; _lr_geo_location=BR; fp_id=WEB:d138bbf648060ceedacf6752e7d10c38; cf_clearance=K34.129qI0Dz1ZD258DEfgDLbUB9y1TIrqFC_Mq1GjU-1704829422-0-2-e6e72e19.bf644945.b1e671f9-0.2.1704829422; TestAB_Groups=gtm-web-vitals_control.adv-cookieless-ppid_enabled.sanityweb50_A.ngage-chat-on-gallery_enabled.payg-discount-re-julius_ml-ranges.ppc-myplan-redirect-lp_enabled.adfacelift-default-web_enabled.ppf-free-insertion-re_enabled.ds-header-navbar_enabled.delivery-quote-weight-11060_control.delivery-quote-weight-3040_control.txp-expanded-delivery-method_enabled.adfacelift-re-web_enabled.billing-history_enabled.sa-price-variation_enabled.ppf-standalone-cars_essential.pay-now-always_enabled.mes-ads-free_enabled.cmod-security-central-my-ads_configA.imo-xp-linkshelf-autocomplete_enabled.ngage-chat-miniprofile_enabled.ppf-boost-motos_control.ngage-adview-miniprofile_enabled.delivery-address-management_a.bumpds-bjperformancetrackb_control.pos-cars-free-edition_control.ppf-edition-re_enabled.chatmod-logged-user-showphonebody_enabled.free-insertion-goods-parcela_control.adsxp-adviewlinks-bjQTZ-3020_control.contentmod-gallery-tip_final.payg-redirect-nc_control.imo-xp-privatead_enabled.adv-desk-redesign_enabled.pos-cars-fee-boost_d-first.adv-listing-lazyload_margin-500.free-edtion-goods-parcela_enabled.payg-discount-julius_ml-c-mab.ngage-listing-adcard-clean_enabled.txp-home-moda-beleza_enabled.fee-boost-goods-parcela_control.autospp-notshow-modal-hv-myads_enabled.adfacelift-goods-web_enabled.ngage-new-home-autos_enabled.autospp-marketplace-header_enabled.adv-amz-call_control.adv-cookieless-liveramp_enabled.additional-seller-info-profile_control.rec-home-by-region_on.additional-seller-info-edit_control; __cf_bm=BF0fuyMQ4EuS8T0m1YpfaXJq0oP.qcF6QeGuUQFKFlg-1705329250-1-ARNpPLA5SnYw+4WK4jMQDp1vSDSZKxWnU09L0Je/hiLLuPFR314HXhSjHgbUzCZ0PlwSw6d1REwkgQh8LMRS4IA=; _cfuvid=iVMLA9OCKNpUbu91JYOHObtsUyzYBaRl3r5sZaFF.rI-1705329251590-0-604800000; mf_b837e449-83ee-457f-9ef5-8f976953f2bc=||1705329252775||0||||0|0|27.1292; s_id=6240c020-3736-448f-84c1-e35752526b342024-01-15T14:34:12.828Z; _gid=GA1.3.1000907121.1705329253; panoramaId_expiry=1705934071609; panoramaId=9aa19ae6329db998ac32688ab1d4185ca02c31ab9399332e2a94603c75a32e6d; panoramaIdType=panoDevice; nvg83482=1305aaca77778a873f60c10a7a10|0_16; olx-cookies-accept=1; pubaccid=4faa46c4-016c-4b2a-8e48-2f6a6fbf42dd; _ga=GA1.3.1658291985.1689600385; cto_bundle=noh7Z194VjZhenB5djMxUjZaa0hmTjBTNGNERUY3ZWpGSExHTWNCJTJCYzVTS1k5eiUyQjVpRUtXdTlZZmE5T1ZEUFFDJTJCRGxzWll5S2hFRmdSY0h4SzljTlU4aWFwQmhyME5LenAzWGdiY2pXb0ZIazNLd1pqVjJabzdGMFN6dExwME83dCUyQlklMkJEQXdqamQxVTN3ZEZXVVFBJTJGa0s5d1ElM0QlM0Q; _ga_50C013M2CC=GS1.1.1705329253.82.1.1705329843.21.0.0; _dd_s=rum=0&expire=1705330742872',
            'referer': 'https://www.olx.com.br/estado-pa',
            'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"',
            'sec-fetch-dest': 'document',
            'sec-fetch-mode': 'navigate',
            'sec-fetch-site': 'same-origin',
            'sec-fetch-user': '?1',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
        }

        return requests.request("GET", url, headers=headers)

In [4]:
olx = OLX()

In [5]:
res = olx.request_olx('https://www.olx.com.br/servicos/estado-pa?sct=9')

In [6]:
res.content

b'<!DOCTYPE html>\n<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->\n<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->\n<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->\n<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->\n<head>\n<title>Attention Required! | Cloudflare</title>\n<meta charset="UTF-8" />\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />\n<meta http-equiv="X-UA-Compatible" content="IE=Edge" />\n<meta name="robots" content="noindex, nofollow" />\n<meta name="viewport" content="width=device-width,initial-scale=1" />\n<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/cf.errors.css" />\n<!--[if lt IE 9]><link rel="stylesheet" id=\'cf_styles-ie-css\' href="/cdn-cgi/styles/cf.errors.ie.css" /><![endif]-->\n<style>body{margin:0;padding:0}</style>\n\n\n<!--[if gte IE 10]><!-->\n<script>\n  if (!navigator.cookieEnabled) {\n    window.addEventListen